```
Copyright 2024 DeepMind Technologies Limited

All software is licensed under the Apache License, Version 2.0 (Apache 2.0);
you may not use this file except in compliance with the Apache 2.0 license.
You may obtain a copy of the Apache 2.0 license at:
https://www.apache.org/licenses/LICENSE-2.0

All other materials are licensed under the Creative Commons Attribution 4.0
International License (CC-BY). You may obtain a copy of the CC-BY license at:
https://creativecommons.org/licenses/by/4.0/legalcode

Unless required by applicable law or agreed to in writing, all software and
materials distributed here under the Apache 2.0 or CC-BY licenses are
distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND,
either express or implied. See the licenses for the specific language governing
permissions and limitations under those licenses.

This is not an official Google product.
```

# Output Decompositions from AlphaTensor-Quantum

This Colab shows how to load and inspect the decompositions provided along with the AlphaTensor-Quantum paper. Decompositions are provided in `.npz` files, each containing a dictionary mapping from the target circuit name to a Numpy array of shape `(num_decompositions, num_factors, tensor_size)`, where we report *at most* `num_decompositions = 10` non-equivalent decompositions (equivalence is determined solely based on factor permutations).

In [ ]:
import numpy as np

from google.colab import files

## Example 1: Loading factorizations without gadgets

All the decompositions in those files whose name end with `_no_gadgets` do not contain any gadgets, i.e., there is a direct one-to-one mapping between the number of factors and the T-count.

As an example, run the following block. When prompted, upload the file named `benchmarks_no_gadgets.npz`.

In [ ]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]
with open(filename, 'rb') as f:
  decompositions = np.load(f)

In [ ]:
# Print the names of all the targets in this file.
print(list(decompositions.keys()))

Now, we obtain the decompositions for the target `'qft_4'`, which has `43` qubits (after compilation) and for which the reported T-count (without gadgets) is `53`.

In [ ]:
# Obtain the decompositions for the target of interest
target_circuit_name = 'qft_4'  # @param {type: "string"}
expected_size = 43  # @param {type: "integer"}
expected_tcount = 53  # @param {type: "integer"}

with open(filename, 'rb') as f:
  decompositions = np.load(f)
  # We need a deep copy as the file will be closed after the `open` block.
  factors = np.copy(decompositions[target_circuit_name]).astype(np.int32)

num_decompositions, num_factors, size = factors.shape
print(f'{num_decompositions=}   {num_factors=}   {size=}')

# Verify the T-count and the size.
assert size == expected_size
assert num_factors == expected_tcount

# Verify that all the non-equivalent `num_decompositions` decompositions give
# the same tensor.
tensors = np.mod(np.einsum('bru,brv,brw->buvw', factors, factors, factors), 2)
for tensor in tensors:
  np.testing.assert_array_equal(tensor, tensors[0])

## Example 2: Loading factorizations with gadgets

All the decompositions in those files whose name does not end with `_no_gadgets` may contain Toffoli and CS gadgets. Due to the presence of gadgets, there is not a one-to-one correspondence between the number of factors and the T-count.

As an example, run the following block. When prompted, upload the file named `benchmarks_gadgets.npz`.

In [ ]:
uploaded = files.upload()
filename = list(uploaded.keys())[0]
with open(filename, 'rb') as f:
  decompositions = np.load(f)

In [ ]:
# Print the names of all the targets in this file.
print(list(decompositions.keys()))

Now, we obtain the decompositions for the target `'qft_4'`, which has `43` qubits (after compilation) and for which the reported T-count (with gadgets) is `44`. Specifically, it contains `4` Toffoli gadgets and `3` CS gadgets.

In [ ]:
# Obtain the decompositions for the target of interest
target_circuit_name = 'qft_4'  # @param {type: "string"}
expected_size = 43  # @param {type: "integer"}
expected_tcount = 44  # @param {type: "integer"}
expected_num_toffoli_gadgets = 4  # @param {type: "integer"}
expected_num_cs_gadgets = 3  # @param {type: "integer"}

with open(filename, 'rb') as f:
  decompositions = np.load(f)
  # We need a deep copy as the file will be closed after the `open` block.
  factors = np.copy(decompositions[target_circuit_name]).astype(np.int32)

num_decompositions, num_factors, size = factors.shape
print(f'{num_decompositions=}   {num_factors=}   {size=}')

# Verify the T-count and the size.
assert size == expected_size
assert num_factors >= expected_tcount

# Verify that all the non-equivalent `num_decompositions` decompositions give
# the same tensor.
tensors = np.mod(np.einsum('bru,brv,brw->buvw', factors, factors, factors), 2)
for tensor in tensors:
  np.testing.assert_array_equal(tensor, tensors[0])

Verify that the T-count and the number of gadgets coincide with the quantities reported in the paper.

In [ ]:
#@title Function to find gadgets

def _check_cs_gadget(factors: np.ndarray) -> bool:
  a, b, ab = factors
  linearly_independent = np.any(a != b)
  linear_dependencies = np.all(ab == np.mod(a + b, 2))
  return linearly_independent and linear_dependencies

def _check_toffoli_gadget(factors: np.ndarray) -> bool:
  a, b, c, ab, ac, abc, bc = factors
  linearly_independent = (
      np.any(a != b) and np.any(a != c) and np.any(b != c) and
      np.any(c != np.mod(a + b, 2))
  )
  linear_dependencies = (
      np.all(ab == np.mod(a + b, 2)) and
      np.all(ac == np.mod(a + c, 2)) and
      np.all(abc == np.mod(a + b + c, 2)) and
      np.all(bc == np.mod(b + c, 2))
  )
  return linearly_independent and linear_dependencies

def find_gadgets(
    factors: np.ndarray
) -> tuple[int, int, int, np.ndarray, np.ndarray]:
  """Finds the gadgets present in the input factorization.

  Args:
    factors: The input factorization, as an array of shape (num_factors, size).
      It is assumed that none of the input factors is the all-zero factor.

  Returns:
    A 5-tuple containing:
    - The effective T-count of the factorization.
    - The number of Toffoli gadgets.
    - The number of CS gadgets.
    - Whether each factor is part of a Toffoli gadget, as a boolean array of
      shape (num_factors,).
    - Whether each factor is part of a CS gadget, as a boolean array of
      shape (num_factors,).
  """
  num_factors, _ = factors.shape
  num_toffoli_gadgets = 0
  num_cs_gadgets = 0
  factors_in_toffoli_gadget = np.zeros((num_factors,), dtype=np.bool_)
  factors_in_cs_gadget = np.zeros((num_factors,), dtype=np.bool_)

  for r in range(num_factors):
    completed_toffoli = False
    completed_cs = False
    factors_not_in_gadgets = np.logical_not(
        np.logical_or(factors_in_toffoli_gadget, factors_in_cs_gadget)
    )
    # Check for a Toffoli gadget.
    if (r >= 6 and _check_toffoli_gadget(factors[(r - 6):(r + 1)]) and
        np.all(factors_not_in_gadgets[(r - 6):(r + 1)])):
      completed_toffoli = True
      num_toffoli_gadgets += 1
      factors_in_toffoli_gadget[(r - 6):(r + 1)] = True
    # Check for a CS gadget.
    if (r >= 2 and _check_cs_gadget(factors[(r - 2):(r + 1)]) and
        np.all(factors_not_in_gadgets[(r - 2):(r + 1)])):
      completed_cs = True
      num_cs_gadgets += 1
      factors_in_cs_gadget[(r - 2):(r + 1)] = True
    # Sanity checks.
    assert not (completed_toffoli and completed_cs)
    assert num_toffoli_gadgets == np.sum(factors_in_toffoli_gadget) // 7
    assert num_cs_gadgets == np.sum(factors_in_cs_gadget) // 3
    assert not np.any(
        np.logical_and(factors_in_toffoli_gadget, factors_in_cs_gadget)
    )

  # Obtain the equivalent T-count.
  tcount = num_factors - 5 * num_toffoli_gadgets - num_cs_gadgets
  return (
      tcount, num_toffoli_gadgets, num_cs_gadgets,
      factors_in_toffoli_gadget, factors_in_cs_gadget
  )

In [ ]:
for f in factors:
  tcount, num_toffoli_gadgets, num_cs_gadgets, _, _ = find_gadgets(f)
  assert tcount == expected_tcount
  assert num_toffoli_gadgets == expected_num_toffoli_gadgets
  assert num_cs_gadgets == expected_num_cs_gadgets

We can also find out which factors form part of a gadget (in this example, we inspect the first of the decompositions):

In [ ]:
_, _, _, factors_in_toffoli_gadget, factors_in_cs_gadget = find_gadgets(
    factors[0]
)
print(f'{factors_in_toffoli_gadget=}')
print(f'{factors_in_cs_gadget=}')